# Interactive Simplicial Complex from Information Theory Analysis of Attention

This notebook first compares the $1$-skeleton of the simplicial complex obtained from the pairwise Jensen-Shannon divergence (this may need to be changed to a distance metric by taking its square root) between the softmax of the attention mechanism applied to the input tokens. So, it:

1. Computes the softmax of the attention mechanism applied to the image input tokens to obtain probability distributions $P(X_i)$. 

2. Compute the Jensen-Shannon distance between each pair of distributions obtained from the token. 

3. Run a persistent homology analysis of the distance matrix. 

4. Plot the $1$-skeleton of the simplicial complexes at a threshold defined by a slider. 

In [1]:
pip install timm -q

Note: you may need to restart the kernel to use updated packages.


In [2]:
import numpy as np
import torch
from transformers import ViTModel, ViTFeatureExtractor
import gudhi as gd
import matplotlib.pyplot as plt
import networkx as nx
from scipy.spatial.distance import jensenshannon
import plotly.graph_objs as go
from PIL import Image
import ipywidgets as widgets


def get_attention_matrix(image, model, tokenizer, layer, head):
    inputs = tokenizer(images=[image], return_tensors="pt")
    outputs = model(**inputs, output_attentions=True)
    attention = outputs.attentions[layer][0, head].detach().cpu().numpy()
    return attention

def compute_persistence(attention_matrix):
    softmax_attention = np.exp(attention_matrix) / np.sum(np.exp(attention_matrix), axis=-1)[:, np.newaxis]
    distance_matrix = np.array([[jensenshannon(softmax_attention[i], softmax_attention[j]) for j in range(softmax_attention.shape[0])] for i in range(softmax_attention.shape[0])])
    
    rips_complex = gd.RipsComplex(distance_matrix=distance_matrix, max_edge_length=np.inf)
    simplex_tree = rips_complex.create_simplex_tree(max_dimension=2)
    persistence = simplex_tree.persistence(min_persistence=0.01)
    return persistence, simplex_tree, distance_matrix

def plot_simplicial_complex_3d(simplex_tree, distance_matrix, title, threshold, tokens):
    g = nx.Graph()
    for (simplex, _) in simplex_tree.get_filtration():
        if len(simplex) == 2:
            if distance_matrix[simplex[0]][simplex[1]] <= threshold:
                g.add_edge(simplex[0], simplex[1])

    # Create a token dictionary only for nodes in the graph
    labels = {node: tokens[node] for node in g.nodes()}
    
    # 3D layout
    pos = nx.spring_layout(g, dim=3, seed=42)
    
    # Extract node coordinates
    Xn = [pos[k][0] for k in g.nodes()]
    Yn = [pos[k][1] for k in g.nodes()]
    Zn = [pos[k][2] for k in g.nodes()]
    
    # Extract edge coordinates
    Xe = []
    Ye = []
    Ze = []
    for e in g.edges():
        Xe += [pos[e[0]][0], pos[e[1]][0], None]
        Ye += [pos[e[0]][1], pos[e[1]][1], None]
        Ze += [pos[e[0]][2], pos[e[1]][2], None]
    
    # Create a trace for edges
    trace_edges = go.Scatter3d(x=Xe, y=Ye, z=Ze, mode='lines', line=dict(color='gray', width=1))
    
    # Create a trace for nodes
    trace_nodes = go.Scatter3d(x=Xn, y=Yn, z=Zn, mode='markers+text', text=list(labels.values()), marker=dict(symbol='circle', size=10, color='lightblue'), textposition="top center")
    
    # Create a layout
    layout = go.Layout(title=title, scene=dict(xaxis_title='X', yaxis_title='Y', zaxis_title='Z'), showlegend=False)
    
    # Create a plot
    fig = go.Figure(data=[trace_edges, trace_nodes], layout=layout)
    fig.show()

# Load pre-trained visual transformer model and tokenizer
tokenizer = ViTFeatureExtractor.from_pretrained('google/vit-base-patch16-224')
model = ViTModel.from_pretrained('google/vit-base-patch16-224')

# Image inputs
image1 = Image.open("heart.jpg")
image2 = Image.open("platonic_metatron.jpg")

# Choose a layer and head
layer = 1
head = 2

# Get attention matrices
attention_matrix1 = get_attention_matrix(image1, model, tokenizer, layer, head)
attention_matrix2 = get_attention_matrix(image2, model, tokenizer, layer, head)

# Compute persistence and simplex trees
persistence1, simplex_tree1, distance_matrix1 = compute_persistence(attention_matrix1)
persistence2, simplex_tree2, distance_matrix2 = compute_persistence(attention_matrix2)

# Get tokens as patch indices
tokens1 = list(range(attention_matrix1.shape[0]))
tokens2 = list(range(attention_matrix2.shape[0]))

# Create a slider for the threshold value
threshold_slider = widgets.FloatSlider(
    value=0.001,
    min=0.00001,
    max=0.05,
    step=0.00001,
    description='Threshold:',
    continuous_update=False
)

# Define a function to update the plot based on the threshold value from the slider
def update_plot(threshold):
    plot_simplicial_complex_3d(simplex_tree1, distance_matrix1, "Simplicial Complex for Image 1", threshold, tokens1)
    plot_simplicial_complex_3d(simplex_tree2, distance_matrix2, "Simplicial Complex for Image 2", threshold, tokens2)

# Connect the slider to the update_plot function
widgets.interact(update_plot, threshold=threshold_slider)


/Users/amelieschreiber/opt/anaconda3/lib/python3.9/site-packages/transformers/models/vit/feature_extraction_vit.py:28: FutureWarning: The class ViTFeatureExtractor is deprecated and will be removed in version 5 of Transformers. Please use ViTImageProcessor instead.
  warnings.warn(
Some weights of the model checkpoint at google/vit-base-patch16-224 were not used when initializing ViTModel: ['classifier.weight', 'classifier.bias']
- This IS expected if you are initializing ViTModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing ViTModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of ViTModel were not initialized from the model checkpoint at google/vit-base-patch16-224 and are newl

interactive(children=(FloatSlider(value=0.001, continuous_update=False, description='Threshold:', max=0.05, mi…

<function __main__.update_plot(threshold)>